# Preamble

In [1]:
%load_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
import sys 
from os.path import abspath
from pathlib import Path
root_path = Path(abspath('..'))

if root_path not in sys.path: 
    sys.path.insert(0, root_path.as_posix())
print(sys.path[0])

/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/SMFF


In [3]:
#directory to store data in. 
data_path = root_path.joinpath("data")
data_path.mkdir(exist_ok=True)

In [4]:
import subprocess
import matplotlib.pyplot as plt
import os
from copy import deepcopy
import galsim 
import corner 
import numpy as np 

from matplotlib.patches import Ellipse

In [5]:
import corner 

from src import defaults 
from src.analysis import fisher
from src.analysis import gparameters
from src.analysis import models
from src.analysis import images
from src.analysis import readfits

# Galaxy model

In [40]:
#parameters of galaxy to draw. 
model = 'gaussian' #possible options of model and psf_model are in documentation. 
psf_model = 'gaussianpsf'
g1 = .2
g2 = .2 
hlr = 0.7 
psf_fwhm = .7
snr = 20.
x0 = 1 # arcsecs
y0 = 1.0

#project folder name where you want to write galaxy information 
project = data_path.joinpath('project-fit-center')

#image parameters 
pixel_scale=.2 # arcsecs/pixel
nx = 40
ny = 40

In [41]:
cmd = (f'cd {root_path.as_posix()}; python -m src.generate -p {project.as_posix()} '
       f'-gal 1 --galaxy-model {model} --psf_model {psf_model}  --g1 {g1} --g2 {g2} --y0 {y0} --x0 {x0} '
       f'--flux 1. --psf_flux 1. --hlr {hlr} --psf_fwhm {psf_fwhm} --snr {snr}')
subprocess.run(cmd, shell=True);

# Do a fits

In [42]:
g_parameters = gparameters.GParameters(project) #extract parameters from file created by generate.py 
image_renderer = images.ImageRenderer(pixel_scale=pixel_scale,
                                      nx=nx,ny=ny)
fish = fisher.Fisher(g_parameters=g_parameters,image_renderer=image_renderer, 
                     snr=snr)

In [43]:
g_parameters.id_params

{'1': OrderedDict([('galaxy_model', 'gaussian'),
              ('psf_model', 'gaussianpsf'),
              ('hlr', 0.7),
              ('flux', 1.0),
              ('x0', 1.0),
              ('y0', 1.0),
              ('g2', 0.2),
              ('g1', 0.2),
              ('psf_fwhm', 0.7),
              ('psf_flux', 1.0)])}

In [63]:
# want to move the center around a little bit inside the pixel to avoid bias when fitting. 
id_params = deepcopy(g_parameters.id_params)
gal_id = next(iter(id_params)) #obtain first key in id_params (there is only one)
id_params[gal_id]['x0'] +=  np.random.uniform() * pixel_scale
id_params[gal_id]['y0'] +=  np.random.uniform() * pixel_scale
new_g_parameters = gparameters.GParameters(id_params=id_params)

new_g_parameters.id_params

{'1': OrderedDict([('galaxy_model', 'gaussian'),
              ('psf_model', 'gaussianpsf'),
              ('hlr', 0.7),
              ('flux', 1.0),
              ('x0', 1.0673017532504012),
              ('y0', 1.0123476505070916),
              ('g2', 0.2),
              ('g1', 0.2),
              ('psf_fwhm', 0.7),
              ('psf_flux', 1.0)])}

In [65]:
orig_image = fish.model.drawImage(nx=defaults.NX, ny=defaults.NX, scale=defaults.PIXEL_SCALE)
mins = defaults.get_minimums(g_parameters, orig_image)
maxs = defaults.get_maximums(g_parameters, orig_image)
init_values = defaults.get_initial_values_fit(g_parameters)

In [64]:
from src import runfits

results = runfits.perform_fit(new_g_parameters, image_renderer, snr=1000, noise_seed=np.random.randint(999999))
results

In [ ]:
gal_id = next(iter(id_params)) #obtain first key in id_params (there is only one)
id_params[gal_id]['hlr'] = hlr
g_parameters = gparameters.GParameters(id_params=id_params)